In [ ]:
# starting with "standard" merge-containing solution
# no prior edges have been inspected

import pandas as pd


ds_summary_pth = '/home/ddon0001/PhD/experiments/scaled/no_div_constraint_err_seg/summary.csv'
ds_summary = pd.read_csv(ds_summary_pth)

ds_edge_class_pth = '/home/ddon0001/PhD/experiments/scaled/no_div_constraint_err_seg/all_edges_with_target_ws_fa_fe.csv'
ds_edge_class = pd.read_csv(ds_edge_class_pth)

ds_with_err = []
for ds_name in ds_summary['ds_name'].unique():
    ds_edges = ds_edge_class[ds_edge_class['ds_name'] == ds_name]
    if ds_edges.solution_incorrect.sum() != 0:
        ds_with_err.append(ds_name)


In [ ]:
from tracktour._tracker import VirtualVertices
from tracktour_experiments.generate_configs import get_config_for_row

out_pth = '/home/ddon0001/PhD/experiments/scaled/pre-thesis/ducb_w_resolve'

def populate_label_ws_enter_exit(
        all_edges,
        solution_graph,
        gt_graph,
        sol_to_gt
    ):
    def is_edge_tp(edge):
        u, v = int(edge['u']), int(edge['v'])
        # appearance edge, correct if destination node in gt_graph
        # has incoming degree 0
        if u == VirtualVertices.APP.value:
            return int(gt_graph.in_degree(sol_to_gt[v]) == 0)
        # exit edge, correct if source node in gt_graph
        # has outgoing degree 0
        elif v == VirtualVertices.TARGET.value:
            return int(gt_graph.out_degree(sol_to_gt[u]) == 0)
        elif not solution_graph.has_edge(u, v):
            return 0
        return int('EdgeFlag.CTC_FALSE_POS' not in solution_graph.edges[u, v]) and 'EdgeFlag.WRONG_SEMANTIC' not in solution_graph.edges[u, v]

    def get_error_cat(edge):
        u, v = int(edge['u']), int(edge['v'])
        if u == VirtualVertices.APP.value:
            if edge['flow'] <= 0:
                return 'None'
            if not edge['oracle_is_correct']:
                return 'FA'
            return 'Correct'
        if v == VirtualVertices.TARGET.value:
            if edge['flow'] <= 0:
                return 'None'
            if not edge['oracle_is_correct']:
                return 'FE'
            return 'Correct'
        if not solution_graph.has_edge(u, v):
            return 'None'
        if 'EdgeFlag.CTC_FALSE_POS' not in solution_graph.edges[u, v]:
            return 'FP'
        if 'EdgeFlag.WRONG_SEMANTIC' not in solution_graph.edges[u, v]:
            return 'WS'
        return 'Correct'

    all_edges['oracle_is_correct'] = all_edges.apply(is_edge_tp, axis=1)
    all_edges['error_type'] = all_edges.apply(get_error_cat, axis=1)

for ds_name in ds_with_err:
    ds_summary_row = ds_summary[ds_summary['ds_name'] == ds_name].squeeze()
    initial_config = get_config_for_row(
        ds_summary_row,
        out_root=out_pth,
        div_constraint=False,
    )
    # solve, keep tracked
    tracked = initial_config.run()
    # evaluate
    results, matched = initial_config.evaluate(tracked.tracked_detections, tracked.tracked_edges)
    gt_to_sol = {item[0]: item[1] for item in matched.mapping}
    sol_to_gt = {item[1]: item[0] for item in matched.mapping}
    # assign edge error types
    populate_label_ws_enter_exit(
        tracked.all_edges,
        matched.pred_graph.graph,
        matched.gt_graph.graph,
        sol_to_gt
    )
    print(len(tracked.all_edges))

    # get ranked edges for each feature
    # assign DUCB rank to each edge?
    # sample n edges and correct errors
    # evaluate & save
    # fix inspected edges
    # re-solve & save
    # evaluate & save
    # update the config...? 
    # repeat until we've inspected all edges?

    
